# Saving intermediate model weights with keras callback

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler #remove the mean and variance
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')

file=r'D:\Subscribers (1).csv'

mydata=pd.read_csv(file)

predictors=mydata.drop(['Subscribers'],1)# removes specific row or coloumn

In [2]:
target=mydata['Subscribers']

In [4]:
x_train,x_test,y_train,y_test=train_test_split(predictors,
                                               target,test_size=0.2,
                                          random_state=2)

In [5]:
scaler = StandardScaler().fit(x_train)

sd_x_train=scaler.transform(x_train)
sd_x_test=scaler.transform(x_test)




In [6]:
cw = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)

TypeError: compute_class_weight() takes 1 positional argument but 3 were given

In [7]:
class_weight_dict = dict(enumerate(cw))
class_weight_dict

NameError: name 'cw' is not defined

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout,Dense


In [ ]:
model=Sequential()

model.add(Dense(30,input_dim=50,activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(20, activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(10,activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(20,activation='tanh'))

model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

In [ ]:
outputFolder = './subscriber_model_output'

In [ ]:
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

In [ ]:
filepath=outputFolder+"/weights-{epoch:02d}-{accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=True, 
                             mode='auto', period=10)

# this will save the weights every 10 epoch

# Early Stopping 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
earlystop = EarlyStopping(monitor='loss', 
                          min_delta=0.01, patience=5,
                          verbose=1, mode='auto')

# Custom Callbacks

In [ ]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

In [ ]:
class my_custom_callback(Callback):
    # There are multiple actions that you can take at various stages of model fitting process
    # Those functions are commented out right now, but if you want you can add actions
    # I have added a auc score calcualtion at every epoch's end
    
    def __init__(self,features,target):
        self.test_data=(features,target)
    
    
        

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.test_data[0])
        auc=roc_auc_score(self.test_data[1], y_pred)
        print(' \n auc score for validation data :',auc,'\n')
        return
    
#     def on_train_begin(self,logs={}):
#         return
#     def on_train_end(self, logs={}):
#         return

#     def on_epoch_begin(self, epoch, logs={}):
#         return

#     def on_batch_begin(self, batch, logs={}):
#         return

#     def on_batch_end(self, batch, logs={}):
#         return

In [ ]:
mycallback=my_custom_callback(sd_x_test,y_test)

In [ ]:
model.fit(sd_x_train,y_train.values,
          class_weight=class_weight_dict,
          epochs=200,
          batch_size=10000,verbose=1,
          callbacks=[mycallback,earlystop,checkpoint]
         )